<a href="https://colab.research.google.com/github/josenomberto/UTEC-CDIAV3-MISTI/blob/main/day4_intro_to_model_training_convolutional_neural_networks_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**If you haven't already, please hit :**

`File` -> `Save a Copy in Drive`

**to copy this notebook to your Google drive, and work on a copy. If you don't do this, your changes won't be saved!**

# Model Training with Convolutional Neural Networks using MNIST

In [ ]:
# @title
# ==============================================
# 0. Module imports
# ==============================================

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets, transforms

# data manipulation
import pandas as pd
import numpy as np
import scipy.stats as st
import scipy

# plots
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pylab as pl

# classification algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler

# dimension reduction
from sklearn.decomposition import PCA

# cross-validation
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn import model_selection

# parallel processing
from joblib import Parallel, delayed
import multiprocessing

# model evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix, classification_report

import warnings
warnings.filterwarnings("ignore")

import seaborn as sns

device = "cpu"

Some visualization code. No need to understand this cell, but don't forget to run it!

In [ ]:
# @title
from graphviz import Digraph

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#93D30C", "#8F00FF"]
def visualize(model):
    hidden_layers = []
    output_layer = 0
    layers = [layer for layer in model.modules() if type(layer) == torch.nn.Linear]

    # custom script to get all the shapes of a network's layers
    sizes = list()
    state_dict = net.state_dict()
    for idx, key in enumerate(state_dict.keys()):
      if key.endswith('weight'):
        output_size, input_size = state_dict[key].shape
        if idx == 0:
          sizes.append(input_size)
        sizes.append(output_size)

    input_layer = sizes[0]
    for layer in layers:
        if layer == layers[0]:
            hidden_layers.append(layer.out_features)

        else:
            if layer == layers[-1]:
                output_layer = layer.out_features
            else:
                hidden_layers.append(layer.out_features)

        last_layer_nodes = input_layer
        nodes_up = input_layer

    g = Digraph("g", filename='visualization_tmp')
    n = 0
    g.graph_attr.update(splines="false", nodesep="0.5", ranksep="0", rankdir='LR')
    # Input Layer
    with g.subgraph(name="cluster_input") as c:
          c.attr(color="white")
          for i in range(0, sizes[0]):
              n += 1
              c.node(str(n))
              c.attr(labeljust="1")
              c.attr(label="Input Layer", labelloc="bottom")
              c.attr(rank="same")
              c.node_attr.update(
                  width="0.65",
                  style="filled",
                  shape="circle",
                  color=HAPPY_COLORS_PALETTE[3],
                  fontcolor=HAPPY_COLORS_PALETTE[3],
              )
    for layer_idx in range(0, len(sizes) - 2):
        with g.subgraph(name="cluster_" + str(layer_idx + 1)) as c:
            c.attr(color="white")
            c.attr(rank="same")
            label = f'Hidden Layer {layer_idx + 1}'
            c.attr(labeljust="right", labelloc="b", label=label)
            for j in range(0, hidden_layers[layer_idx]):
                n += 1
                c.node(
                    str(n),
                    width="0.65",
                    shape="circle",
                    style="filled",
                    color=HAPPY_COLORS_PALETTE[0],
                    fontcolor=HAPPY_COLORS_PALETTE[0],
                )
                for h in range(nodes_up - last_layer_nodes + 1, nodes_up + 1):
                    g.edge(str(h), str(n))
            last_layer_nodes = hidden_layers[layer_idx]
            nodes_up += hidden_layers[layer_idx]

    with g.subgraph(name="cluster_output") as c:
        c.attr(color="white")
        c.attr(rank="same")
        c.attr(labeljust="1")
        for i in range(1, output_layer + 1):
            n += 1
            c.node(
                str(n),
                width="0.65",
                shape="circle",
                style="filled",
                color=HAPPY_COLORS_PALETTE[4],
                fontcolor=HAPPY_COLORS_PALETTE[4],

            )
            for h in range(nodes_up - last_layer_nodes + 1, nodes_up + 1):
                g.edge(str(h), str(n))
        c.attr(label="Output Layer", labelloc="bottom")
        c.node_attr.update(
            color="#2ecc71", style="filled", fontcolor="#2ecc71", shape="circle"
        )

    g.attr(arrowShape="none")
    g.edge_attr.update(arrowhead="none", color="#707070", penwidth="2")
    g.view()
    return g

## Define DataLoaders

This code creates **data loaders** for the **MNIST dataset** using PyTorch, which handles the loading and batching of training and testing data. The **batch sizes** are set to 64 for training and 1000 for testing to balance computation efficiency and memory usage. The `train_loader` shuffles the training data for better generalization, while the `test_loader` keeps the test data in a fixed order.

Both data loaders apply a series of **transformations** to the images: converting them to tensors and normalizing pixel values to have a mean of 0.1307 and a standard deviation of 0.3081. These normalization values are calculated based on the MNIST dataset and help improve model convergence during training.

In [ ]:
# Define the Batch Sizes
batch_size_train = 64
batch_size_test = 1000

# Training DataLoader
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('sample_data/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

# Testing DataLoader
test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('sample_data/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=False)

### ANALYZE: Explore Common Neural Network Layers

In this exercise, we explore common PyTorch layers and use them to transform an example MNIST image.

Tasks:
1. Explore `nn.Flatten()` - Converts a multi-dimensional tensor into a single-dimensional array.
2. Explore `nn.Linear()`– Applies a linear transformation to the input.
3. Explore `nn.ReLU()`– Introduces non-linearity to the network.
4. Explore `nn.Sequential()`– Combines layers into a sequence.
5. Explore `nn.Softmax()`Converts raw scores (logits) into probabilities.

In [ ]:
data_iter = iter(train_loader)
images, labels = next(data_iter)
input_image = images[0]
print(input_image.size())

torch.Size([1, 28, 28])


#### TASK 1: nn.Flatten

The `nn.Flatten()` layer reshapes a 2D image into a 1D array of pixel values. This is often the first step when using fully connected layers, as they require a flat input.

In [ ]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# Original image
ax[0].imshow(input_image.squeeze(), cmap='gray')
ax[0].set_title(f'Original Image - Label: {labels[0].item()}')
ax[0].axis('off')

# Plot the flattened Image
ax[1].imshow(flat_image.detach().numpy(), cmap='gray', aspect='auto')
ax[1].set_title('Flattened Image (784 pixels)')
ax[1].set_xlabel('Pixel Index')
ax[1].set_ylabel('Pixel Value')

sns.despine()

plt.tight_layout()
plt.show()

#### TASK 2: nn.Linear

The `nn.Linear()` layer applies a linear transformation to the input data using its weights and biases. This helps the network create a feature space.

In [ ]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# Original image
ax[0].imshow(input_image.squeeze(), cmap='gray')
ax[0].set_title(f'Original Image - Label: {labels[0].item()}')
ax[0].axis('off')

# Hidden layer output as a bar chart
ax[1].bar(range(hidden1.size(1)), hidden1.detach().numpy().squeeze())
ax[1].set_title('Hidden Layer Output (20 features)')
ax[1].set_xlabel('Feature Index')
ax[1].set_ylabel('Feature Value')

sns.despine()

plt.tight_layout()
plt.show()

#### TASK 3: nn.ReLU

The `nn.ReLU()` activation function introduces non-linearity into the model. Without non-linear functions, the network would be limited to learning only linear relationships.

In [ ]:
hidden1_ReLU = nn.ReLU()(hidden1)

fig, ax = plt.subplots(1, 2, figsize=(12, 6), sharey = True)

# Original image
ax[0].bar(range(hidden1.size(1)), hidden1.detach().numpy().squeeze())
ax[0].set_title(f'Original Image - Label: {labels[0].item()}')

# Hidden layer output as a bar chart after ReLU
ax[1].bar(range(hidden1_ReLU.size(1)), hidden1_ReLU.detach().numpy().squeeze())
ax[1].set_title('Hidden Layer Output (ReLU Applied, 20 Features)')
ax[1].set_xlabel('Feature Index')
ax[1].set_ylabel('Feature Value')

sns.despine()

plt.tight_layout()
plt.show()

#### TASK 4: nn.Sequential

The `nn.Sequential()` module allows us to define a complete network by chaining multiple layers together. This creates a straightforward, ordered flow of data through the network.

In [ ]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
logits = seq_modules(input_image)

# Capture outputs at each step
flattened = flatten(input_image)
layer1_output = layer1(flattened)
relu_output = nn.ReLU()(layer1_output)
logits = seq_modules(input_image)

# Plot the outputs
fig, axes = plt.subplots(1, 4, figsize=(15, 4))

axes[0].imshow(input_image.squeeze().numpy(), cmap='gray')
axes[0].set_title("Original Image")

axes[1].imshow(flattened.detach().numpy(), cmap='gray', aspect='auto')
axes[1].set_title("Flattened")

axes[2].bar(range(20), layer1_output.detach().numpy().squeeze())
axes[2].set_title("Layer 1 Output")

axes[3].bar(range(10), logits.detach().numpy().squeeze())
axes[3].set_title("Logits (Output)")

sns.despine()

plt.tight_layout()
plt.show()

#### TASK 5: nn.Softmax

The `nn.Softmax()` layer converts raw logits into probabilities that sum to 1. It’s typically applied to the final layer of a classification network to produce class probabilities.

In [ ]:
# Apply Softmax to convert logits to probabilities
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

# Plot the logits and the resulting probabilities
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Logits (Before Softmax)
axes[0].bar(range(10), logits.squeeze().detach().numpy())
axes[0].set_title("Logits (Before Softmax)")
axes[0].set_xlabel("Class Index")
axes[0].set_ylabel("Value")

# Predicted Probabilities (After Softmax)
axes[1].bar(range(10), pred_probab.squeeze().detach().numpy())
axes[1].set_title("Predicted Probabilities (After Softmax)")
axes[1].set_xlabel("Class Index")
axes[1].set_ylabel("Probability")

sns.despine()

plt.tight_layout()
plt.show()

## Training a Simple Convolutional Network

The `train()` function is responsible for updating the network's parameters during the **training phase** using data from the training set. It sets the network to **training mode**, enabling layers like dropout that improve generalization. For **fully connected networks**, the input data is flattened into a 1D vector before being passed to the network, but this step is **skipped for convolutional neural networks (CNNs)**, which handle multi-dimensional inputs directly. During each batch, the function performs a **forward pass** to make predictions, calculates the **loss** using the Negative Log Likelihood function, and performs **backpropagation** to update the network’s weights. The function tracks and logs the **training progress** to monitor performance over time.

In [ ]:
# Note: network.train() puts the network into "nondeterministic mode".
# This doesn't matter for the fully connected network, but it will matter for convolutional networks.
# CNNs rely on preserving spatial structure, so the nondeterministic mode can impact results.
def train(network, optimizer, epoch, trainloader, flatten_data=True):

    network.train()  # Set the network to training mode
    loss_logs = list()  # Track the loss over time

    for batch_idx, (data, target) in enumerate(trainloader):
        # For fully connected networks, we flatten the image data to feed it into the network.
        # CNNs, however, expect multi-dimensional image inputs, so we skip flattening.
        if flatten_data:
            data = flatten(data)  # Flatten the image (only for fully connected networks)

        # Move data to GPU if available
        data, target = data.to(device), target.to(device)

        # Zero the gradients from the previous batch
        optimizer.zero_grad()

        # Perform the forward pass through the network
        output = network(data)

        # Compute the loss using the Negative Log Likelihood loss function
        # This loss function is often used for classification tasks with multiple classes.
        loss = F.nll_loss(output, target)

        # Backpropagation: Calculate the gradients of the loss with respect to the network's parameters
        loss.backward()

        # Perform the optimization step: Update the network's parameters
        optimizer.step()

        # Print training progress at regular intervals
        if batch_idx % log_interval == 0:
            examples_shown = batch_idx * len(data)
            total_examples = len(trainloader.dataset)
            fraction_shown = round(examples_shown / total_examples * 100.0, 2)
            rounded_loss = round(loss.item(), 4)
            print(f'Train Epoch {epoch} Progress: {fraction_shown}%\tLoss: {rounded_loss}')

            examples_so_far = batch_idx * len(data) + epoch * len(trainloader.dataset)
            loss_logs.append((examples_so_far, loss.item()))

    return loss_logs

This `test()` function evaluates the performance of a trained neural network on unseen data from the test set. It puts the network into **evaluation mode** to ensure consistent behavior by disabling layers like dropout, which are only active during training. The function then loops through the test data, performs a **forward pass** to generate predictions, and calculates both the **average loss** and **accuracy**. For fully connected networks, the input data must be **flattened into a 1D vector** before being passed to the network, but this step is unnecessary for **convolutional neural networks (CNNs)**, which process multi-dimensional image data directly. The function finally prints the results, helping us assess how well the model generalizes to new data.

In [ ]:
def test(network, flatten_data=True):
    # Set the network to evaluation mode.
    # This disables certain behaviors like dropout that are only used during training.
    network.eval()

    # Initialize test loss and correct predictions counters
    test_loss = 0
    correct = 0

    # Disable gradient calculation for efficiency since we are only testing the model.
    with torch.no_grad():
        for data, target in test_loader:
            # For fully connected networks, flatten the image data into a 1D vector.
            # CNNs expect multi-dimensional image inputs, so flattening is unnecessary for them.
            if flatten_data:
                data = flatten(data)

            # Move data and target to the GPU if available
            data, target = data.to(device), target.to(device)

            # Perform a forward pass through the network
            output = network(data)

            # Calculate the cumulative test loss using Negative Log Likelihood loss
            test_loss += F.nll_loss(output, target, size_average=False).item()

            # Get the predicted class for each input sample
            pred = output.data.max(1, keepdim=True)[1]

            # Count the number of correct predictions
            correct += pred.eq(target.data.view_as(pred)).sum()

    # Calculate the average test loss by dividing by the number of samples
    test_loss /= len(test_loader.dataset)

    # Calculate the test accuracy as a percentage
    accuracy = round(100. * correct.item() / len(test_loader.dataset), 2)

    # Print the test loss and accuracy
    print(f'\nTest set: Avg loss {round(test_loss, 4)}, Accuracy {accuracy}%\n')

    # Return the test loss and accuracy for logging purposes
    return test_loss, accuracy

### EXERCISE: Define a Convolutional Neural Network

Tasks:
1. Define a convolutional network with the following architecture.

  - Define the CNN layers.
    
    A. Replace `<REPLACE ME AND MY ARROWS>` with the correct code to define the second convolutional layer using `nn.Conv2d()`.
      - Input channels = 1 (grayscale image)
      - Output channels = 10
      - Kernel size = 5
    
    B. Replace `<REPLACE ME AND MY ARROWS>` with the correct code to define the second convolutional layer using `nn.Conv2d()`.
      - Input channels = 10 (from the previous layer's output)
      - Output channels = 20
      - Kernel size = 5
    
    C. Replace `<REPLACE ME AND MY ARROWS>` with the correct code to add a dropout layer using `nn.Dropout2d()`.
    
    D. Replace `<REPLACE ME AND MY ARROWS>` with the correct code to define the first fully connected layer using `nn.Linear()`.
      - Input features = 320 (flattened feature maps)
      - Output features = 50
    
    E. Replace `<REPLACE ME AND MY ARROWS>` with the correct code to define the second fully connected layer using `nn.Linear()`.
      - Input features = 50 (from the previous fully connected layer)
      - Output features = 10 (number of classes)
    
  - Define the forward pass of the CNN.

    F. Replace `<REPLACE ME AND MY ARROWS>` with the correct code to apply the first convolutional layer followed by ReLU activation and 2x2 max pooling.
    
    G. Replace `<REPLACE ME AND MY ARROWS>` with the correct code to apply the second convolutional layer followed by a dropout layer.
    
    H. Replace `<REPLACE ME AND MY ARROWS>` with the correct code to apply 2x2 max pooling to the output of the second convolutional layer.

    I. Replace `<REPLACE ME AND MY ARROWS>` with the correct code to apply a ReLU activation to introduce non-linearity
    
    J. Replace `<REPLACE ME AND MY ARROWS>` with the correct code to apply a fully connected layer with a ReLU activation.

    K. Replace `<REPLACE ME AND MY ARROWS>` with the correct code to apply a fully connected layer.

In [ ]:
# TASK 1 EXERCISE

# Define the convolutional neural network (CNN) architecture
class ConvNet(nn.Module):

    def __init__(self):
        super(ConvNet, self).__init__()

        # First convolutional layer:
        # - 1 input channel (grayscale image)
        # - 10 output channels (feature maps)
        # - 5x5 kernel size (learns 5x5 filters to detect patterns)
        self.conv1 = <REPLACE ME AND MY ARROWS> # QUESTION A

        # Second convolutional layer:
        # - Takes 10 input feature maps from conv1
        # - Produces 20 output feature maps
        # - 5x5 kernel size
        self.conv2 = <REPLACE ME AND MY ARROWS> # QUESTION B

        # Dropout layer to reduce overfitting by randomly dropping units during training
        self.conv2_drop = <REPLACE ME AND MY ARROWS> # QUESTION C

        # First fully connected layer:
        # - 320 input features (flattened feature maps after convolutions)
        # - 50 output features (reduces dimensionality for classification)
        self.fc1 = <REPLACE ME AND MY ARROWS> # QUESTION D

        # Second fully connected layer:
        # - 50 input features from the previous layer
        # - 10 output features corresponding to the number of classes
        self.fc2 = <REPLACE ME AND MY ARROWS> # QUESTION E

    def forward(self, x):
        # First convolutional layer followed by:
        # - ReLU activation (introduces non-linearity)
        # - Max pooling (reduces dimensionality by taking the max value from 2x2 regions)
        x = <REPLACE ME AND MY ARROWS> # QUESTION F

        # Second convolutional layer followed by dropout:
        # - Helps reduce overfitting by randomly dropping out units
        x = <REPLACE ME AND MY ARROWS> # QUESTION G

        # Apply max pooling to reduce the feature map size
        # - Uses a 2x2 pooling region
        x = <REPLACE ME AND MY ARROWS> # QUESTION H

        # Apply ReLU activation to introduce non-linearity
        x = <REPLACE ME AND MY ARROWS> # QUESTION I

        # Flatten the output to a 1D vector before passing it to fully connected layers
        # - The input size is reshaped to (-1, 320), where -1 infers the batch size
        x = x.view(-1, 320)

        # Apply the first fully connected layer followed by ReLU activation
        x = <REPLACE ME AND MY ARROWS> # QUESTION J

        # Apply dropout to the fully connected layer's output to reduce overfitting
        # - Dropout is only applied during training
        x = F.dropout(x, training=self.training)

        # Apply the second fully connected layer (output layer)
        # - Produces 10 output values corresponding to class scores
        x = <REPLACE ME AND MY ARROWS> # QUESTION K

        # Return the log of softmax probabilities for classification
        # - Converts the output scores into a categorical probability distribution
        return F.log_softmax(x)

### EXERCISE: Train a Convolutional Neural Network with MNIST

Tasks:
1. Set Up Your Environment and Define the Network
  - Define key hyperparameters to control the training process.
  - Create an instance of your CNN model using `ConvNet()`.
  - Move the model to the appropriate device (CPU/GPU).
  - Configure the optimizer using `optim.SGD()`.
2. Train Your CNN Model.
  - Train your model for the specified number of epochs using the `train()` function.
3. Evaluate and Visualize the Model’s Performance.
  - Use the `plot_results()` function to visualize:
    - Training Loss and Test Loss Over Time
    - Test Accuracy Over Epochs
    - Analyze the plots to assess your model's performance.

#### TASK 1: Prepare your Environment to Train your CNN

In [ ]:
# TASK 1 EXERCISE

# Define hyperparameters
# Hyperparameters control the behavior of the training process.

learning_rate = 0.01  # Controls how large the weight updates are during training.
# A smaller learning rate leads to slower but more stable convergence.
# A larger learning rate can speed up convergence but may cause instability.

momentum = 0.9
# Momentum helps accelerate convergence by adding a fraction of the previous update to the current update.
# This helps the model avoid getting stuck in local minima and smooths the optimization process.

n_epochs = 2
# Number of complete passes through the training dataset during training.
# Increasing the number of epochs can improve performance, but may lead to overfitting if too high.

log_interval = 10
# How often (in batches) we print the training progress.
# For example, a log_interval of 10 means the network will print updates every 10 batches.

# Initialize the convolutional network
conv_network = <REPLACE ME AND MY BRACKETS>

# Move the network to the GPU if available
<REPLACE ME AND MY BRACKETS>

# Define the optimizer
# We're using SGD (Stochastic Gradient Descent) with the specified learning rate and momentum.
conv_optimizer = <REPLACE ME AND MY BRACKETS>

#### TASK 2: Train your Model

In [ ]:
# TASK 2 EXERCISE

# Initialize lists to track test accuracy, test losses, and training losses
test_accuracies, test_losses, train_losses = list(), list(), list()
total_examples_seen = 0  # Tracks the total number of examples processed during training

# Training loop over the specified number of epochs
for epoch in range(n_epochs):

    # Run the test phase before training each epoch to log initial performance
    test_loss, test_accuracy = test(conv_network, flatten_data=False)

    # Train the network on the training dataset for the current epoch
    train_loss_logs = <REPLACE ME AND MY ARROWS>

    # Log test performance
    test_accuracies.append((epoch, test_accuracy))  # Log accuracy for this epoch
    test_losses.append((total_examples_seen, test_loss))  # Log test loss

    # Log training losses
    train_losses.extend(train_loss_logs)

    # Update the total number of examples seen
    total_examples_seen = train_loss_logs[-1][0]

# Final test phase after completing all epochs
test_loss, test_accuracy = test(conv_network, flatten_data=False)
test_accuracies.append((n_epochs, test_accuracy))
test_losses.append((total_examples_seen, test_loss))

# At the end of the training process, we have:
# - test_accuracies: A list tracking the model's accuracy on the test set after each epoch.
# - test_losses: A list tracking the test loss after each epoch.
# - train_losses: A detailed log of the training loss over time.

#### TASK 3: Assess the Performance of your Model

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_results(test_accuracies, test_losses, train_losses):
    """
    Plot training losses, test losses, and test accuracy over time.

    Parameters:
    - test_accuracies: List of tuples [(epoch, accuracy)], representing test accuracy over epochs.
    - test_losses: List of tuples [(epoch, loss)], representing test loss over examples seen.
    - train_losses: List of tuples [(example, loss)], representing training loss over examples seen.
    """
    # --- Plot Training and Test Losses ---
    plt.figure(figsize=(12, 6))

    # Extract values for training losses
    train_x, train_y = zip(*train_losses)

    # Extract values for test losses
    test_x, test_y = zip(*test_losses)

    # Plot training losses
    plt.plot(train_x, train_y, label='Training Loss', color='b', linestyle='-', marker='.')

    # Plot test losses
    plt.plot(test_x, test_y, label='Test Loss', color='r', linestyle='--', marker='o')

    # Formatting
    plt.xlabel('Examples Seen')
    plt.ylabel('Log Loss')
    plt.title('Training and Test Losses Over Time')
    plt.legend()
    plt.grid(True)
    plt.show()

    # --- Plot Test Accuracy ---
    plt.figure(figsize=(12, 6))

    # Extract values for test accuracy
    acc_x, acc_y = zip(*test_accuracies)

    # Plot test accuracy
    plt.plot(acc_x, acc_y, label='Test Accuracy', color='g', marker='o')

    # Formatting
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Test Accuracy Over Epochs')
    plt.legend()
    plt.grid(True)
    sns.despine()
    plt.show()


In [ ]:
# TASK 3 EXERCISE

# Plot your Results using the function `plot_resolts
''' ADD YOUR CODE HERE '''

# Appendices

## Hyperparameter Tuning with CNNs

### EXERCISE: Perform Hyperparameter Tuning to Improve Performance of your Convolutional Neural Network



In [ ]:
from itertools import product

# Define ranges for hyperparameters
learning_rates = [0.001, 0.01, 0.1]
momentums = [0.5, 0.9]
batch_sizes = [32, 64, 128]

# Store results
results = []

# Hyperparameter tuning loop
for lr, momentum, batch_size in product(learning_rates, momentums, batch_sizes):
    print(f"Training with lr={lr}, momentum={momentum}, batch_size={batch_size}")

    # Update train loader with the current batch size
    train_loader = torch.utils.data.DataLoader(
        torchvision.datasets.MNIST('sample_data/', train=True, download=True,
                                   transform=torchvision.transforms.Compose([
                                       torchvision.transforms.ToTensor(),
                                       torchvision.transforms.Normalize((0.1307,), (0.3081,))
                                   ])),
        batch_size=batch_size, shuffle=True)

    # Initialize network
    conv_network = ConvNet()
    conv_network.to(device)

    # Define optimizer
    conv_optimizer = optim.SGD(conv_network.parameters(), lr=lr, momentum=momentum)

    # Train and test
    test_accuracies, test_losses, train_losses = list(), list(), list()
    total_examples_seen = 0
    for epoch in range(n_epochs):
        test_loss, test_accuracy = test(conv_network, flatten_data=False)
        train_loss_logs = train(conv_network, conv_optimizer, epoch, train_loader, flatten_data=False)
        test_accuracies.append((epoch, test_accuracy))
        test_losses.append((total_examples_seen, test_loss))
        train_losses.extend(train_loss_logs)
        total_examples_seen = train_loss_logs[-1][0]
    test_loss, test_accuracy = test(conv_network, flatten_data=False)

    # Log the performance
    results.append({
        "lr": lr,
        "momentum": momentum,
        "batch_size": batch_size,
        "test_loss": test_loss,
        "test_accuracy": test_accuracy
    })

In [ ]:
# Print the results sorted by accuracy
sorted_results = sorted(results, key=lambda x: x["test_accuracy"], reverse=True)
for result in sorted_results:
    print(result)

## Insight into Saving Models
A common way to save a model is to serialize the internal state dictionary (containing the model parameters).



In [ ]:
torch.save(conv_network.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## Insight into Loading Models

The process for loading a model includes re-creating the model structure and loading
the state dictionary into it.



In [ ]:
model = ConvNet()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

This model can now be used to make predictions.



In [ ]:
# Download test data from open datasets.
test_data = datasets.MNIST('sample_data/',
    train=False,
    download=True,
    transform=transforms.ToTensor(),
)

classes = [1, 2, 3, 4, 5, 6, 7, 8, 9]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "8", Actual: "8"
